In [44]:
import pandas as pd
import numpy as np
import os, sys, pytz
import matplotlib.pyplot as plt
from tqdm import tqdm
from warnings import filterwarnings
filterwarnings('ignore')

import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from sklearn.metrics import r2_score
from scipy.stats import spearmanr
import wandb
import random
import pickle

import pandas as pd
import numpy as np
import alpaca
from alpaca_trade_api import REST
import matplotlib.pyplot as plt
import datetime
from tqdm import tqdm
import pytz

### get stock data
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
from alpaca.data.historical import StockHistoricalDataClient

import sys
sys.path = sys.path + ['../']

import my_secrets
import pickle

from warnings import filterwarnings
filterwarnings('ignore')

import time

In [45]:
stock_code='TQQQ'

In [46]:
path = f'./historical_price_data/{stock_code}.csv'
data = pd.read_csv(path,index_col=0)
data.index = pd.Series([i.replace(tzinfo=None) for i in pd.to_datetime(data.index)]).dt.tz_localize(pytz.timezone('US/Eastern'))
a = data.resample('1min').mean().dropna()
a['change_rate'] = (a['close'] - a['open'])/a['open']
a['change_bi'] = np.where((a['close'] - a['open'])>0,1,0)
a['date'] = pd.to_datetime([i.date() for i in a.index])
a['year'] = a.date.dt.year
a['WOY'] = a.date.dt.week
a['WOY_year'] = [str(a)+str(b) for a,b in zip(a['WOY'],a['year'])]
a['MOY'] = a.date.dt.month
a['MOY_year'] = [str(a)+str(b) for a,b in zip(a['MOY'],a['year'])]
a['DOW'] = a.date.dt.day_of_week
a['DOY'] = a.date.dt.day_of_year
a['DOY_year'] = [str(a)+str(b) for a,b in zip(a['DOY'],a['year'])]
a['MOD'] = [i.hour*60+i.minute for i in a.index]
a['HOD'] = [i.hour for i in a.index]
a = a[(a.MOD>=9.5*60) & (a.MOD<=16*60)]
a['change_rate'] = (a['close'] - a['open'])/a['open']
whole_sequence = a['change_rate'].values
del a


In [47]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
class myloader():
    def __init__(self, whole_sequence, past_feature_size=150000, batch_size=10):
        self.whole_sequence = whole_sequence
        self.past_feature_size = past_feature_size
        self.batch_size = batch_size
        self.total = int(len(whole_sequence)/batch_size)
    
    def get_data(self):
        batch_size = self.batch_size
        ## random idx
        
        idx_pool = list(range(len(self.whole_sequence)))[self.past_feature_size+10:]
        np.random.shuffle(idx_pool)
        
        x_list = []
        y_list = []
        for idx in idx_pool:
            if idx<=self.past_feature_size+10:
                continue
            else:
                
                x = np.expand_dims(
                    self.whole_sequence[idx-self.past_feature_size:idx].reshape(self.past_feature_size,-1),
                    axis=0
                )
                x = torch.tensor(x).float()*100
                y = torch.tensor(self.whole_sequence[idx]).reshape(1,-1).float()*100
                x_list.append(x)
                y_list.append(y)
                
                if not len(x_list)>=batch_size:
                    continue
                else:
                    x = torch.cat(x_list)
                    y = torch.cat(y_list)
                    x_list = []
                    y_list = []
                    yield x,y
            

In [49]:
INPUT_SEQUENCE_LENGTH = 5000

In [50]:
5000/60/6.5

12.82051282051282

In [51]:
len(whole_sequence[0:int(len(whole_sequence)*0.6)])

309266

In [52]:
len(whole_sequence[int(len(whole_sequence)*0.65):])

180406

In [53]:
len(whole_sequence)

515444

In [54]:
train_loader = myloader(whole_sequence[0:int(len(whole_sequence)*0.6)],
                        INPUT_SEQUENCE_LENGTH,
                        batch_size=50
                       )
val_loader = myloader(whole_sequence[int(len(whole_sequence)*0.65):],
                        INPUT_SEQUENCE_LENGTH,
                      batch_size=50
                       )


In [55]:
class StockPricePredictionModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1, dropout=0.2):
        super(StockPricePredictionModel, self).__init__()
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(input_size, nhead=1, dim_feedforward=hidden_size, dropout=dropout, batch_first=True),
            num_layers
        )
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        x = self.transformer_encoder(x)
        x = self.fc1(x[:,-1,:])
        x = nn.functional.relu(x)
        x = self.fc2(x)
        return x

In [56]:
input_size = 1  # Number of features for each minute
hidden_size = 64  # Size of the hidden layer
output_size = 1  # Predicting one value: the minute-level price change
num_layers = 1  # Number of transformer encoder layers
dropout = 0.2  # Dropout probability
model = StockPricePredictionModel(input_size, hidden_size, output_size, num_layers, dropout)
model = model.to('cpu').float()


In [58]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()
for epoch in range(100):
    
    #### train
    model.train()
    train_losses = []
    val_losses = []
    optimizer.zero_grad()
    for batch_count,(x,y) in tqdm(enumerate(train_loader.get_data()), total = train_loader.total):
        if batch_count>100:
            break
        preds = model(x)
        loss = criterion(preds, y)
        loss.backward()
        with torch.no_grad():
            train_losses.append(loss.item())
        optimizer.step()
#         print(f'loss: {loss.item():.4f}')
    
    ### eval
    model.eval()
    for batch_count,(x,y) in tqdm(enumerate(val_loader.get_data()), total = val_loader.total):
        if batch_count>100:
            break
        preds = model(x)
        loss = criterion(preds, y)
        with torch.no_grad():
            val_losses.append(loss.item())
        
    print(f"Epoch {epoch+1}, train_loss: {np.nanmean(train_losses)}, val_loss: {np.nanmean(val_losses)}")
    
    

  0%|          | 11/3608 [01:54<10:23:25, 10.40s/it]


Epoch 1, train_loss: 0.033312346786260605, val_loss: 0.04829886132343249


  0%|          | 11/3608 [01:53<10:21:06, 10.36s/it]


Epoch 2, train_loss: 0.03831720521504229, val_loss: 0.04056826470927759


  0%|          | 3/6185 [02:35<88:57:50, 51.81s/it]


KeyboardInterrupt: 